# Triangulation Sidequest

This technique for trianglulation including internal points turned out to be unnessisary after switching to `shapely.constrained_delaunay_triangles`

In [ ]:
import numpy as np
import shapely as sh
import matplotlib.pyplot as plt
from airfoil.util.array_helpers import split_indexable
from airfoil.util.shapely_helpers import (
    plot_shapely_directional,
    plot_shapely
)
from airfoil.util.linestring_helpers import (
    deflection_angle,
    resample_spline_fallback_linear,
    ensure_closed,
    remove_sequential_duplicates,
)

In [ ]:
fig, (
    (axa, axb, axc),
    (axd, axe, axf),
) = plt.subplots(2,3,figsize=(15,10))


# create components
s0 = sh.Point(0,0).buffer(60)
s1 = sh.Point(0,0).buffer(60+30)
s2 = sh.box(-100,-100,100,0)
s3 = sh.box(-15,50, 15,250)
plot_shapely_directional([s0,s1,s2,s3], ax=axa)
axa.set_title("components")


# use boolean operations to create spork shape
axb.set_title("polygon")
spork:sh.Polygon = sh.difference(sh.union(s3,sh.difference(s1,s2)),s0)
plot_shapely_directional([spork],legend=["spork"],ax=axb)


# split polygon at its corners into individual segments
axc.set_title("split at corners")
segments:list[np.ndarray] = split_indexable(
    spork.exterior.coords, 
    np.where(deflection_angle(spork.exterior.coords)>np.deg2rad(30))[0]+1
)
plot_shapely_directional([sh.LineString(segment) for segment in segments], ax=axc)


# resample the edges so that they have points at a nice consistent interval
axd.set_title("re-sampled\n(using scipy.make_splprep if possible,\nfalling back to linear interpolation when this fails)")
resampled = [resample_spline_fallback_linear(segment, lambda total_length:np.ceil(total_length/5)) for segment in segments]
plot_shapely_directional([sh.LineString(segment) for segment in resampled], ax=axd)

# connect the segments back into a polygon
axe.set_title("recombined")
spork_edge = sh.Polygon(remove_sequential_duplicates(ensure_closed(np.concat(resampled))))
plot_shapely_directional([spork_edge], ax=axe)

#triangulate the polygon
axf.set_title("triangulated")
spork_triangulated = sh.MultiPolygon([triangle for triangle in sh.constrained_delaunay_triangles(spork_edge).geoms])
plot_shapely([spork_triangulated],ax=axf)

segment_lengths = [len(segment) for segment in resampled]
print(f"The number of points in each segment at the re-sampled stage were {segment_lengths}")
fig.tight_layout()

In [ ]:
from collections import defaultdict
def filter_close_points(points, min_distance, grid_factor=2.0):
    
    # Grid cell size - make it larger than min_distance to reduce checks
    cell_size = min_distance * grid_factor
    
    # Find bounding box and create grid coordinates
    min_coords = points.min(axis=0)
    grid_coords = ((points - min_coords) / cell_size).astype(int)
    
    # Group points by grid cell
    grid = defaultdict(list)
    for i, coord in enumerate(grid_coords):
        grid[tuple(coord)].append(i)
    
    # Track which points to keep
    keep = np.ones(len(points), dtype=bool)
    
    # Generate offsets for adjacent cells (including current cell)
    # For d dimensions, check all 3^d neighboring cells
    d = points.shape[1]
    offsets = np.array(np.meshgrid(*[[-1, 0, 1]] * d)).T.reshape(-1, d)
    
    # Process each grid cell
    for cell_coord, point_indices in grid.items():
        if not point_indices:
            continue
            
        # Collect points from this cell and adjacent cells
        candidates = []
        for offset in offsets:
            neighbor_coord = tuple(np.array(cell_coord) + offset)
            if neighbor_coord in grid:
                candidates.extend(grid[neighbor_coord])
        
        # Remove duplicates and sort
        candidates = sorted(set(candidates))
        
        # Check distances between all pairs in this neighborhood
        for i, idx1 in enumerate(candidates):
            if not keep[idx1]:
                continue
                
            for idx2 in candidates[i+1:]:
                if not keep[idx2]:
                    continue
                    
                # Calculate distance
                dist = np.linalg.norm(points[idx1] - points[idx2])
                if dist < min_distance:
                    # Remove the second point (arbitrary choice)
                    keep[idx2] = False
    
    return points[keep]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,5))
x_range,y_range = np.array(spork_edge.bounds).reshape(-1,2).transpose()

pnt = np.asarray([
    np.random.uniform(*x_range,10000),
    np.random.uniform(*y_range,10000),
]).transpose()
pnt_mask = [spork_edge.contains(point:=sh.Point(*item)) and sh.distance(point, spork_edge.exterior)>3 for item in pnt]
pnt = pnt[pnt_mask]
pnt = filter_close_points(pnt, min_distance=5)
ax=plot_shapely_directional([spork_edge], ax=ax1)
ax.plot(*pnt.transpose(),".")
meesh = sh.MultiPolygon([
    t
    for t
    in sh.delaunay_triangles(sh.GeometryCollection([sh.Point(item) for item in pnt]+[spork_edge])).geoms
    if spork_edge.contains(t)
])
plot_shapely([meesh],ax=ax2)